Import Libraries and Load Data

In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv("/dataset.csv")

checking first 5 columns

In [29]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


checking for errors in the dataset

In [30]:
df.isnull().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


Combine date and time into a single datetime column

In [32]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

Sort by Timestamp and Parking Lot ID

In [33]:
df.sort_values(by=['Timestamp', 'SystemCodeNumber'], inplace=True)

Calculate Occupancy Ratio

In [34]:
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

Initialize a price column (starting with base price = $10)

In [35]:
df['Price'] = 10.0

Baseline Linear Price Function

In [36]:
def linear_price_model(prev_price, occupancy_ratio, alpha=2.0):
    return prev_price + alpha * occupancy_ratio

Store the current price for each lot

In [37]:
current_prices = {}

In [38]:
calculated_prices = []

In [39]:
for idx, row in df.iterrows():
    lot_id = row['SystemCodeNumber']
    occupancy_ratio = row['OccupancyRatio']

Get previous price or use base price

In [40]:
prev_price = current_prices.get(lot_id, 10.0)

Calculate new price

In [41]:
 new_price = linear_price_model(prev_price, occupancy_ratio)

Keep price within smooth range

In [42]:
 new_price = max(5.0, min(new_price, 20.0))

Update current price for the lot

In [43]:
current_prices[lot_id] = new_price

Save the new price

In [44]:
calculated_prices.append(new_price)

In [45]:
current_prices = {}
calculated_prices = []

for idx, row in df.iterrows():
    lot_id = row['SystemCodeNumber']
    occupancy_ratio = row['OccupancyRatio']

    # Get previous price or use base price
    prev_price = current_prices.get(lot_id, 10.0)

    # Calculate new price
    new_price = linear_price_model(prev_price, occupancy_ratio)

    # Keep price within smooth range
    new_price = max(5.0, min(new_price, 20.0))

    # Update current price for the lot
    current_prices[lot_id] = new_price

    # Save the new price
    calculated_prices.append(new_price)

In [46]:
df['Price_Model_1'] = calculated_prices

In [47]:
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'OccupancyRatio', 'Price_Model_1']].head(10)

,SystemCodeNumber,Timestamp,Occupancy,Capacity,OccupancyRatio,Price_Model_1
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,0.105719,10.211438
1312,BHMBCCTHL01,2016-10-04 07:59:00,120,387,0.310078,10.620155
2624,BHMEURBRD01,2016-10-04 07:59:00,117,470,0.248936,10.497872
3936,BHMMBMMBX01,2016-10-04 07:59:00,264,687,0.384279,10.768559
5248,BHMNCPHST01,2016-10-04 07:59:00,237,1200,0.197500,10.395000
6560,BHMNCPNST01,2016-10-04 07:59:00,249,485,0.513402,11.026804
7872,Broad Street,2016-10-04 07:59:00,178,690,0.257971,10.515942
9184,Others-CCCPS105a,2016-10-04 07:59:00,709,2009,0.352912,10.705824
10496,Others-CCCPS119a,2016-10-04 07:59:00,195,2803,0.069568,10.139137
11808,Others-CCCPS135a,2016-10-04 07:59:00,1081,3883,0.278393,10.556786


 Model 2 (Demand-Based Pricing)

 Assign Weights to Vehicle Types

In [48]:
vehicle_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}

df['VehicleTypeWeight'] = df['VehicleType'].str.lower().map(vehicle_weights)

In [55]:
def compute_demand(occupancy, capacity, queue_length, traffic_level, is_special_day, vehicle_type_weight,
                   alpha=0.4, beta=0.3, gamma=0.2, delta=0.2, epsilon=0.3):
    # Map traffic condition to numerical value
    traffic_mapping = {'low': 0, 'average': 1, 'high': 2}
    traffic_level_numeric = traffic_mapping.get(traffic_level.lower(), 0) # Default to 0 if not found

    # Normalize input features
    occupancy_ratio = occupancy / capacity

    return (alpha * occupancy_ratio +
            beta * queue_length -
            gamma * traffic_level_numeric +
            delta * is_special_day +
            epsilon * vehicle_type_weight)

Calculate Price Based on Normalized Demand

In [50]:
def demand_based_price(base_price, demand, lambd=0.6):
    norm_demand = min(max(demand / 10, 0), 1)
    price = base_price * (1 + lambd * norm_demand)
    return max(0.5 * base_price, min(price, 2 * base_price))

In [51]:
base_price = 10.0
current_prices_model2 = {}
prices_model2 = []

In [56]:
base_price = 10.0
current_prices_model2 = {}
prices_model2 = []

for idx, row in df.iterrows():
    lot_id = row['SystemCodeNumber']

    # Extract features
    occupancy = row['Occupancy']
    capacity = row['Capacity']
    queue_length = row['QueueLength']
    traffic = row['TrafficConditionNearby']
    is_special_day = row['IsSpecialDay']
    vehicle_weight = row['VehicleTypeWeight']

    # Calculate demand
    demand = compute_demand(occupancy, capacity, queue_length, traffic,
                             is_special_day, vehicle_weight)

    # Calculate price from demand
    price = demand_based_price(base_price, demand)

    # Store latest price
    current_prices_model2[lot_id] = price
    prices_model2.append(price)

df['Price_Model_2'] = prices_model2
display(df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'OccupancyRatio', 'Price_Model_1', 'Price_Model_2']].head(10))

,SystemCodeNumber,Timestamp,Occupancy,Capacity,OccupancyRatio,Price_Model_1,Price_Model_2
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,0.105719,10.211438,10.385373
1312,BHMBCCTHL01,2016-10-04 07:59:00,120,387,0.310078,10.620155,10.614419
2624,BHMEURBRD01,2016-10-04 07:59:00,117,470,0.248936,10.497872,10.599745
3936,BHMMBMMBX01,2016-10-04 07:59:00,264,687,0.384279,10.768559,10.632227
5248,BHMNCPHST01,2016-10-04 07:59:00,237,1200,0.197500,10.395000,10.497400
6560,BHMNCPNST01,2016-10-04 07:59:00,249,485,0.513402,11.026804,10.663216
7872,Broad Street,2016-10-04 07:59:00,178,690,0.257971,10.515942,10.601913
9184,Others-CCCPS105a,2016-10-04 07:59:00,709,2009,0.352912,10.705824,10.624699
10496,Others-CCCPS119a,2016-10-04 07:59:00,195,2803,0.069568,10.139137,10.376696
11808,Others-CCCPS135a,2016-10-04 07:59:00,1081,3883,0.278393,10.556786,10.606814


 Model 3 (Competitive Pricing)

In [57]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2.0)**2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

 Precompute Nearby Lots Within 1 km

In [58]:
from collections import defaultdict

lots = df[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates()
nearby_map = defaultdict(list)

for i, row1 in lots.iterrows():
    for j, row2 in lots.iterrows():
        if row1['SystemCodeNumber'] != row2['SystemCodeNumber']:
            dist = haversine(row1['Latitude'], row1['Longitude'],
                             row2['Latitude'], row2['Longitude'])
            if dist <= 1.0:  # within 1 km
                nearby_map[row1['SystemCodeNumber']].append(row2['SystemCodeNumber'])

In [59]:
competitive_prices = []

for idx, row in df.iterrows():
    lot_id = row['SystemCodeNumber']
    timestamp = row['Timestamp']
    base_price = 10.0

    # Get current lot's own demand-based price from Model 2
    own_price = row['Price_Model_2']

    # Get occupancy and capacity
    occupancy = row['Occupancy']
    capacity = row['Capacity']
    occupancy_ratio = occupancy / capacity

    # Check nearby lots
    nearby_lots = nearby_map[lot_id]

    # Get prices of nearby lots at the same timestamp
    nearby_prices = df[(df['SystemCodeNumber'].isin(nearby_lots)) &
                       (df['Timestamp'] == timestamp)][['SystemCodeNumber', 'Price_Model_2']]

    if not nearby_prices.empty:
        avg_nearby_price = nearby_prices['Price_Model_2'].mean()

        # Adjust own price based on competitive situation
        if occupancy_ratio > 0.9 and avg_nearby_price < own_price:

            new_price = own_price * 0.95
        elif avg_nearby_price > own_price:

            new_price = own_price * 1.05
        else:
            new_price = own_price
    else:
        new_price = own_price

    new_price = max(0.5 * base_price, min(new_price, 2 * base_price))

    competitive_prices.append(new_price)

df['Price_Model_3'] = competitive_prices

In [60]:
df[['SystemCodeNumber', 'Timestamp', 'Price_Model_1', 'Price_Model_2', 'Price_Model_3']].head(10)

,SystemCodeNumber,Timestamp,Price_Model_1,Price_Model_2,Price_Model_3
0,BHMBCCMKT01,2016-10-04 07:59:00,10.211438,10.385373,10.385373
1312,BHMBCCTHL01,2016-10-04 07:59:00,10.620155,10.614419,10.614419
2624,BHMEURBRD01,2016-10-04 07:59:00,10.497872,10.599745,10.599745
3936,BHMMBMMBX01,2016-10-04 07:59:00,10.768559,10.632227,10.632227
5248,BHMNCPHST01,2016-10-04 07:59:00,10.395000,10.497400,11.022270
6560,BHMNCPNST01,2016-10-04 07:59:00,11.026804,10.663216,10.663216
7872,Broad Street,2016-10-04 07:59:00,10.515942,10.601913,10.601913
9184,Others-CCCPS105a,2016-10-04 07:59:00,10.705824,10.624699,10.624699
10496,Others-CCCPS119a,2016-10-04 07:59:00,10.139137,10.376696,10.376696
11808,Others-CCCPS135a,2016-10-04 07:59:00,10.556786,10.606814,10.606814


In [61]:
!pip install pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [62]:
df = df.sort_values('Timestamp')
df.to_csv('streaming_dataset.csv', index=False)

In [63]:
import pathway as pw

In [64]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: int
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    Queue_Length: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str
    ID: int
@pw.udf
def compute_price_model_1(occupancy: int, capacity: int, prev_price: float) -> float:
    ratio = occupancy / capacity
    return round(max(5.0, min(prev_price + 2.0 * ratio, 20.0)), 2)

In [66]:
def build_pipeline():
    # Ingest from CSV as streaming source
    table = pw.io.csv.read(
        "streaming_dataset.csv",
        schema=ParkingSchema,
        mode="streaming",
        autocommit_duration_ms=1000,
    )

    # Combine date and time
    table += pw.apply(
        lambda date, time: f"{date} {time}",
        table.LastUpdatedDate, table.LastUpdatedTime
    ).as_column("CombinedTimestamp")

    # Convert to datetime
    table += table.with_columns(
        TimestampParsed=pw.apply(lambda t: pd.to_datetime(t), table.CombinedTimestamp)
    )

    # Add a price column starting with $10
    table += table.with_columns(
        BasePrice=pw.make_constant_column(10.0)
    )

    # Apply Model 1 pricing logic
    table += table.with_columns(
        Price_Model_1=pw.apply(compute_price_model_1,
                               table.Occupancy,
                               table.Capacity,
                               table.BasePrice)
    )

    # Output pricing table
    pw.io.json.write(table.select(
        table.SystemCodeNumber,
        table.TimestampParsed,
        table.Occupancy,
        table.Capacity,
        table.Price_Model_1
    ))

# Run it
if __name__ == "__main__":
    pw.run()

Output()

 Real-Time Pricing with Model 2 (Demand-Based) in Pathway


In [67]:
vehicle_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}

@pw.udf
def get_vehicle_weight(vehicle_type: str) -> float:
    return vehicle_weights.get(vehicle_type.lower(), 1.0)

@pw.udf
def compute_demand(occupancy: int, capacity: int, queue: int, traffic: float, special: int, vehicle_weight: float,
                   alpha=0.4, beta=0.3, gamma=0.2, delta=0.2, epsilon=0.3) -> float:
    occ_ratio = occupancy / capacity
    demand = (alpha * occ_ratio + beta * queue - gamma * traffic + delta * special + epsilon * vehicle_weight)
    return demand

@pw.udf
def compute_price_model_2(base_price: float, demand: float, lambd=0.6) -> float:
    norm_demand = min(max(demand / 10, 0), 1)
    price = base_price * (1 + lambd * norm_demand)
    return round(max(0.5 * base_price, min(price, 2 * base_price)), 2)


In [68]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: int
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    Queue_Length: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str
    ID: int

In [70]:
def build_pipeline():
    table = pw.io.csv.read(
        "streaming_dataset.csv",
        schema=ParkingSchema,
        mode="streaming",
        autocommit_duration_ms=1000,
    )

    table += pw.apply(
        lambda date, time: f"{date} {time}",
        table.LastUpdatedDate, table.LastUpdatedTime
    ).as_column("CombinedTimestamp")

    table += table.with_columns(
        TimestampParsed=pw.apply(lambda t: pd.to_datetime(t), table.CombinedTimestamp),
        BasePrice=pw.make_constant_column(10.0),
        VehicleTypeWeight=pw.apply(get_vehicle_weight, table.VehicleType),
        Demand=pw.apply(
            compute_demand,
            table.Occupancy,
            table.Capacity,
            table.Queue_Length,
            table.TrafficConditionNearby,
            table.IsSpecialDay,
            pw.column("VehicleTypeWeight")
        ),
    )

    table += table.with_columns(
        Price_Model_2=pw.apply(compute_price_model_2, table.BasePrice, table.Demand)
    )

    pw.io.json.write(table.select(
        table.SystemCodeNumber,
        table.TimestampParsed,
        table.Price_Model_2,
        table.Occupancy,
        table.Queue_Length,
        table.TrafficConditionNearby,
        table.VehicleType,
        table.IsSpecialDay
    ))

if __name__ == "__main__":
    pw.run()

Output()

Real-Time Pricing Visualization with Bokeh

In [71]:
!pip install bokeh

In [72]:
sample_lots = df['SystemCodeNumber'].unique()[:3]
plot_df = df[df['SystemCodeNumber'].isin(sample_lots)]

In [73]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Legend
from bokeh.palettes import Category10

In [75]:
output_notebook()

p = figure(title="Dynamic Parking Price Over Time", x_axis_type='datetime',
           x_axis_label='Timestamp', y_axis_label='Price ($)',
           width=800, height=400)

legend_items = []
colors = Category10[10]

for i, lot_id in enumerate(sample_lots):
    lot_data = plot_df[plot_df['SystemCodeNumber'] == lot_id]
    source = ColumnDataSource(data={
        'x': lot_data['Timestamp'],
        'y': lot_data['Price_Model_2'],
    })
    line = p.line(x='x', y='y', source=source, line_width=2, color=colors[i],
                  legend_label=f"Lot {lot_id}")

p.legend.title = 'Parking Lot ID'
p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)

 Conclusion

This project demonstrates a functional real-time pricing engine for urban parking based solely on Python, Pandas, NumPy, and Pathway. With a smart combination of economic theory, data processing, and geolocation analysis, this approach can be deployed in smart cities to optimize parking efficiency, reduce congestion, and improve user experience.